In [ ]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Importar dados
ticker = 'BOVA11.SA'
data = yf.download(ticker, start='2018-01-02', end='2025-05-31', auto_adjust=False)
data.columns = data.columns.get_level_values(0)

In [ ]:

def calcular_retorno_estrategia(data, strategy='buy_hold'):
    
    if strategy == 'buy_hold':
        # Buy and Hold - sempre comprado
        returns = data['Adj Close'].pct_change().dropna()
        
    elif strategy == 'ma_cross':
        # Cruzamento de médias móveis (20 e 50 períodos)
        short_ma = data['Adj Close'].rolling(20).mean()
        long_ma = data['Adj Close'].rolling(50).mean()
        
        # Sinal: 1 quando MA curta > MA longa, 0 caso contrário
        signals = (short_ma > long_ma).astype(int)
        positions = signals.shift(1).fillna(0)  # Executa no próximo dia
        
        market_returns = data['Adj Close'].pct_change()
        returns = (positions * market_returns).dropna()
        
    elif strategy == 'rsi_reversal':
        # RSI Reversal (compra oversold, vende overbought)
        delta = data['Adj Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        
        # Sinais: compra quando RSI < 30, vende quando RSI > 70
        signals = pd.Series(0, index=data.index)
        signals[rsi < 30] = 1  # Compra
        signals[rsi > 70] = 0  # Vende
        
        # Forward fill para manter posições
        signals = signals.replace(0, np.nan).fillna(method='ffill').fillna(0)
        positions = signals.shift(1).fillna(0)
        
        market_returns = data['Adj Close'].pct_change()
        returns = (positions * market_returns).dropna()
    
    else:
        raise ValueError("Estratégia deve ser: 'buy_hold', 'ma_cross', ou 'rsi_reversal'")
    
    return returns

In [ ]:
# Cálculo dos retornos
strategy_name = 'ma_cross'
returns = calcular_retorno_estrategia(data, strategy_name)

In [ ]:
# Cálculo de drawdown histórico
cumulative = (1 + returns).cumprod()
running_max = cumulative.expanding().max()
historical_dd = (cumulative - running_max) / running_max
max_historical_dd = historical_dd.min()

print(f"Drawdown máximo histórico: {abs(max_historical_dd)*100:.2f}%")

In [ ]:
# Simulação de Monte Carlo, assumindo distribuição normal dos retornos

#Parâmetros iniciais
years = 3
num_simulations=10000
initial_value=100000

#Limpar série de retornos
clean_returns = returns.dropna()
clean_returns = clean_returns[clean_returns != 0]

#Número de dias baseado na quantidade de dias úteis no ano
num_days = years * 252

#Inicializar vetores de zeros
max_drawdowns = np.zeros(num_simulations)
final_values = np.zeros(num_simulations)

#Calcula medidas estatísticas dos retornos
mean_return = clean_returns.mean()
std_return = clean_returns.std()

#Iterações para gerar retornos acumulados baseado em escolhas aleatórias de retornos passados
for i in range(num_simulations):

    #Simula retornos aleatoriamente baseado em dados históricos
    simulated_returns = np.random.choice(clean_returns.values, size=num_days, replace=True)
    
    #Cálculo de juros composto
    portfolio_values = initial_value * np.cumprod(1 + simulated_returns)
    
    #Cálculo dos drawdowns e adição dos dados nos vetores de zeros
    running_max = np.maximum.accumulate(portfolio_values)
    drawdowns = (running_max - portfolio_values) / running_max
    max_drawdown = np.max(drawdowns)
    max_drawdowns[i] = max_drawdown
    final_values[i] = portfolio_values[-1]

In [ ]:
#Calcula percentual do drawdown máximo
dd_pct = max_drawdowns * 100

#Visualização
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribuição dos Drawdowns', 'Box Plot', 
                    'Probabilidades Acumuladas', 'Cenários de Risco'),
    specs=[[{"type": "histogram"}, {"type": "box"}],
            [{"type": "scatter"}, {"type": "bar"}]]
)

fig.add_trace(
    go.Histogram(x=dd_pct, nbinsx=50, name="Drawdowns", 
                marker_color='rgba(231, 76, 60, 0.7)'),
    row=1, col=1
)

fig.add_trace(
    go.Box(y=dd_pct, name="Drawdowns", marker_color='rgba(52, 152, 219, 0.7)'),
    row=1, col=2
)

sorted_dd = np.sort(dd_pct)
probabilities = np.arange(1, len(sorted_dd) + 1) / len(sorted_dd) * 100

fig.add_trace(
    go.Scatter(x=sorted_dd, y=probabilities, mode='lines',
                name='CDF', line=dict(color='green', width=2)),
    row=2, col=1
)

percentiles = [50, 75, 90, 95, 99]
values = np.percentile(dd_pct, percentiles)

fig.add_trace(
    go.Bar(x=[f'P{p}' for p in percentiles], y=values,
            name='Percentis', marker_color='rgba(155, 89, 182, 0.7)'),
    row=2, col=2
)

fig.update_layout(
    title=f'Análise Monte Carlo - Drawdowns Futuros<br>{ticker} ({strategy_name})',
    height=800,
    showlegend=False
)

fig.update_xaxes(title_text="Drawdown Máximo (%)", row=1, col=1)
fig.update_xaxes(title_text="Drawdown Máximo (%)", row=2, col=1)
fig.update_xaxes(title_text="Percentil", row=2, col=2)

fig.update_yaxes(title_text="Frequência", row=1, col=1)
fig.update_yaxes(title_text="Drawdown (%)", row=1, col=2)
fig.update_yaxes(title_text="Probabilidade (%)", row=2, col=1)
fig.update_yaxes(title_text="Drawdown (%)", row=2, col=2)

fig.show()

In [ ]:
print("=" * 80)
print(f"RELATÓRIO DE RISCO MONTE CARLO")
print(f"Ativo: {ticker} | Estratégia: {strategy_name}")
print("=" * 80)
print(f"ESTATÍSTICAS DOS DRAWDOWNS FUTUROS (3 anos):")
print(f"   • Drawdown médio esperado: {np.mean(dd_pct):.2f}%")
print(f"   • Drawdown mediano: {np.median(dd_pct):.2f}%")
print(f"   • Desvio padrão: {np.std(dd_pct):.2f}%")
print(f"   • Mínimo (melhor caso): {np.min(dd_pct):.2f}%")
print(f"   • Máximo (pior caso): {np.max(dd_pct):.2f}%")
print(f"\nCENÁRIOS DE RISCO (Probabilidade de EXCEDER):")
risk_levels = [
    (50, "médio"),
    (75, "conservador"), 
    (90, "muito conservador"),
    (95, "extremamente conservador"),
    (99, "catastrófico")
]

for percentile, description in risk_levels:
    value = np.percentile(dd_pct, percentile)
    prob = 100 - percentile
    print(f"   • {prob:2d}% chance de DD > {value:6.2f}% (cenário {description})")

median_dd = np.median(dd_pct)
p95_dd = np.percentile(dd_pct, 95)

if median_dd < 15 and p95_dd < 35:
    risk_class = "BAIXO"
    risk_desc = "Drawdowns moderados esperados"
elif median_dd < 25 and p95_dd < 50:
    risk_class = "MÉDIO"  
    risk_desc = "Drawdowns significativos possíveis"
else:
    risk_class = "ALTO"
    risk_desc = "Drawdowns severos prováveis"

print(f"\nCLASSIFICAÇÃO DE RISCO: {risk_class}")
print(f"   • {risk_desc}")

final_returns = (final_values / initial_value - 1) * 100
positive_scenarios = (final_values > initial_value).mean() * 100

print(f"\nANÁLISE DE RETORNOS (3 anos):")
print(f"   • Probabilidade de lucro: {positive_scenarios:.1f}%")
print(f"   • Retorno médio esperado: {np.mean(final_returns):.2f}%")
print(f"   • Retorno mediano: {np.median(final_returns):.2f}%")
print(f"   • Melhor cenário (P5): +{np.percentile(final_returns, 95):.2f}%")
print(f"   • Pior cenário (P95): {np.percentile(final_returns, 5):.2f}%")

print("=" * 80)

In [ ]:
#Define a curva de capital da estratégia iniciando com R$100000,00 
equity = (1 + returns).cumprod() * 100000

#Percentual do drawdown máximo
drawdown_pct = historical_dd * 100

#Visualização
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(f'Evolução da Carteira - {ticker} ({strategy_name})', 
                    'Drawdown Histórico'),
    vertical_spacing=0.1,
    row_heights=[0.7, 0.3]
)

fig.add_trace(
    go.Scatter(x=equity.index, y=equity.values, 
                mode='lines', name='Valor da Carteira',
                line=dict(color='blue', width=2)),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=drawdown_pct.index, y=drawdown_pct.values,
                mode='lines', name='Drawdown',
                fill='tonexty', fillcolor='rgba(255, 0, 0, 0.3)',
                line=dict(color='red', width=1)),
    row=2, col=1
)

fig.update_layout(
    title=f'Análise Histórica - {ticker}',
    height=600,
    showlegend=False
)

fig.update_yaxes(title_text="Valor (R$)", row=1, col=1)
fig.update_yaxes(title_text="Drawdown (%)", row=2, col=1)
fig.update_xaxes(title_text="Data", row=2, col=1)

fig.add_annotation(
    x=0.02, y=0.95,
    xref="paper", yref="paper",
    text=f"DD Máximo Histórico: {abs(max_historical_dd)*100:.2f}%",
    showarrow=False,
    bgcolor="rgba(255, 255, 255, 0.8)",
    bordercolor="red",
    borderwidth=1
)

fig.show()

O grande problema da simulação de Monte Carlo é partir do principio que o mercado é uma distribuição normal